In [12]:
import requests
import pandas as pd

from bs4 import BeautifulSoup

In [13]:
dataset = pd.read_csv('dota_games.csv', index_col=0)
len(dataset)

5223

In [14]:
for page_id in range(1, 21):
    url = 'http://www.dotabuff.com/matches?page=' + str(page_id)
    header = { 'User-Agent': 'Dota Predictor' }
    request = requests.get(url, headers=header)
    html = request.content
    soup = BeautifulSoup(html)  
    
    if request.status_code == 200:
        print 'parsing page', page_id
    else:
        print 'HTTP error at page', page_id
        print request.status_code
        break
    
    for game_row in soup.select('table tr')[1:]:
        game_cols = game_row.select('td')

        game_data = {
            'match_id': int(game_cols[0].text),
            'game_mode': game_cols[1].text,
            'winner': game_cols[2].text,
            'duration': float(game_cols[3].text.replace(':', '.'))
        }

        if 'match_id' in dataset.columns and any(dataset['match_id'] == game_data['match_id']):
            # print 'skipping match', game_data['match_id']
            continue

        hero_count = 0
        for radient_hero in game_cols[4].find_all('a'):
            hero = radient_hero['href'].split('/')[-1]
            game_data['radient-' + hero] = 1
            hero_count += 1

        for dire_hero in game_cols[5].find_all('a'):
            hero = dire_hero['href'].split('/')[-1]
            game_data['dire-' + hero] = 1
            hero_count += 1
                        
        if hero_count != 10:
            print game_data
            break
            
        dataset = dataset.append(game_data, ignore_index=True)
    
dataset.sort_index(axis=1, inplace=True)
print 'complete'

parsing page 1
parsing page 2
parsing page 3
parsing page 4
parsing page 5
parsing page 6
parsing page 7
parsing page 8
parsing page 9
parsing page 10
parsing page 11
parsing page 12
parsing page 13
parsing page 14
parsing page 15
parsing page 16
parsing page 17
parsing page 18
parsing page 19
parsing page 20
complete


In [15]:
dataset.fillna(0, inplace=True)
dataset['duration'] = dataset['duration'].astype(float)
dataset['match_id'] = dataset['match_id'].astype(int)

In [16]:
dataset.to_csv('dota_games.csv')
len(dataset)

5638

In [17]:
dataset[(dataset['dire-tiny'] == 1) & (dataset['dire-io'] == 1)]['winner'].value_counts()

Radiant Victory    14
Dire Victory       11
dtype: int64